In [1]:
import pandas as pd
import os, ast

from glob import glob
from tqdm import tqdm

In [2]:
import warnings
warnings.filterwarnings('ignore')

## 데이터셋 가져오기

In [3]:
# 각 데이터 가져오기(싱글 판매량/인기순위, 멀티 판매량/인기순위 총 4개)
data_path_list = glob('./find_datas/output/*info.xlsx')
print(len(data_path_list))

4


## 싱글/멀티 판매량인기순위 별 게임의 카테고리 카운트하기

In [4]:
# 데이터별 모든 카테고리의 수 카운트하기
data_dict = {}

# 4개의 데이터 모두 같은 방식을 사용 -> for문 활용
for path in data_path_list:
    # 카테고리 카운트 할 빈 디렉토리 사용
    cat_dict = {}
    # 데이터 만들기
    df = pd.read_excel(path)
    
    for index in df.index:
        # 카테고리 데이터가 리스트 형식의 문자열( '[n,n,...]' )로 되어있기 때문에 ast 라이브러리 이용하여 리스트로 출력
        cats = ast.literal_eval(df.loc[index,'game_category'])
        # 카테고리 카운트
        for cat in cats:
            try:
                # 해당 카테고리가 키값으로 존재하면 거기에 +1
                cat_dict[cat] += 1
            except:
                # 해당 키값이 없으면 키값 생성
                cat_dict[cat] = 1
    
    # 키값으로 쓸 파일 이름 가져오기(os 라이브러리 이용)
    base_name = os.path.basename(path) # 해당 경로의 마지막 부분(파일이름+확장자) 가져오는 함수
    file_name = os.path.splitext(base_name)[0] # 확장자 부분 잘라주는 함수
    
    # 출력용으로 딕셔너리 데이터 정렬  # 리스트타입
    cat_dict2 = sorted(cat_dict.items(), key=lambda x: x[1],reverse=True)
    
    # data_dict에 파일이름을 키값으로 데이터 저장
    data_dict[file_name] = cat_dict
    
    # 파일 이름과 해당 데이터인 cat_dict2 출력
    print(file_name)
    print(cat_dict2[:20])
    print('=='*20)

print('가져온 데이터프레임의 형식: ')
print(df.head(2))

multi_top_rated_info
[('Multiplayer', 107), ('Action', 99), ('Co-op', 82), ('Singleplayer', 73), ('First-Person', 62), ('Online Co-Op', 60), ('Adventure', 56), ('Open World', 48), ('Indie', 47), ('Atmospheric', 44), ('Shooter', 41), ('FPS', 40), ('Simulation', 39), ('Survival', 36), ('Sandbox', 34), ('Strategy', 32), ('PvP', 30), ('Funny', 30), ('Great Soundtrack', 26), ('Tactical', 25)]
multi_top_seller_info
[('Multiplayer', 105), ('Action', 83), ('Singleplayer', 81), ('Co-op', 78), ('Online Co-Op', 54), ('Adventure', 53), ('Open World', 50), ('First-Person', 45), ('Simulation', 37), ('Atmospheric', 37), ('Shooter', 34), ('PvP', 34), ('Sandbox', 34), ('FPS', 31), ('Survival', 31), ('Strategy', 31), ('Third Person', 29), ('RPG', 26), ('Indie', 26), ('Tactical', 25)]
single_top_rated_info
[('Singleplayer', 104), ('Adventure', 85), ('Action', 79), ('Atmospheric', 66), ('Story Rich', 63), ('Great Soundtrack', 54), ('Indie', 47), ('Open World', 43), ('Multiplayer', 37), ('First-Person', 37

## 데이터프레임 만들기

In [5]:
# 딕셔너리를 데이터프레임으로 만들어주기(transpose로 행과 열 바꿔주기)
data_df = pd.DataFrame(data_dict).transpose()

# 가장 보고싶은 컬럼 Multiplayer, Singleplayer 둘을 앞으로 빼주기
MS_df = data_df[['Multiplayer','Singleplayer']].copy()
NMG_df = data_df[data_df.columns.drop(['Multiplayer','Singleplayer'])].copy()
result_df = pd.concat([MS_df,NMG_df],axis = 1)

# Nan 값을 0으로 채워주기
result_df.fillna(0,inplace=True)

In [6]:
# 만들어진 데이터 프레임 중 멀티플레이어 카테고리 인덱스만 가져오기/ Multiplayer 컬럼 지워주기
multi_df = result_df.loc[['multi_top_rated_info','multi_top_seller_info'],data_df.columns.drop(['Multiplayer'])].copy()
multi_df

,FPS,Shooter,Competitive,Action,Team-Based,eSports,Tactical,First-Person,PvP,Online Co-Op,...,NSFW,Dystopian,Gothic,Sequel,Cult Classic,Mouse only,Utilities,Software,Photo Editing,Game Development
multi_top_rated_info,40.0,41.0,15.0,99.0,20.0,8.0,25.0,62.0,30.0,60.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
multi_top_seller_info,31.0,34.0,11.0,83.0,16.0,9.0,25.0,45.0,34.0,54.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# 만들어진 데이터 프레임 중 싱글플레이어 카테고리 인덱스만 가져오기 / Singleplayer 컬럼 지워주기
single_df = result_df.loc[['single_top_rated_info','single_top_seller_info'],data_df.columns.drop(['Singleplayer'])].copy()
single_df

,FPS,Shooter,Multiplayer,Competitive,Action,Team-Based,eSports,Tactical,First-Person,PvP,...,NSFW,Dystopian,Gothic,Sequel,Cult Classic,Mouse only,Utilities,Software,Photo Editing,Game Development
single_top_rated_info,23.0,21.0,37.0,1.0,79.0,2.0,1.0,8.0,37.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
single_top_seller_info,19.0,22.0,63.0,7.0,76.0,2.0,6.0,11.0,30.0,13.0,...,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


## 데이터 분석
### 유클리디안 거리: 피타고라스 정의 이용하여 유사성 체크, 값이 낮을수록(거리가 가까울수록) 유사함  
np.linalg.norm  

In [8]:
import numpy as np

# 변수 만들어주기
a1,b1 = single_df.iloc[0], single_df.iloc[1]
a2,b2 = multi_df.iloc[0], multi_df.iloc[1]

# 싱글 플레이어 판매량/인기순위 간 유사성 구하기
single_dist = np.linalg.norm(a1 - b1)

# 멀티 플레이어 판매량/인기순위 간 유사성 구하기
multi_dist = np.linalg.norm(a2 - b2)

# 출력하기
print(single_dist)
print('=='*10)
print(multi_dist)

91.67333309092672
62.49799996799898


### 상관분석
#### 카테고리 정리하기
데이터프레임의 컬럼(카테고리)은 총 341개이므로 이 중 상관관계가 아주 낮지 않은 것들로 가져온다

In [9]:
# 카운트가 11 미만인 카테고리를 drop_list에 추가한다.
drop_list = []
for column in single_df.columns:
    if single_df[column][0] < 11 and single_df[column][1] < 11:
        drop_list.append(column)
        
drop_list2 = []
for column in multi_df.columns:
    if multi_df[column][0] < 11 and multi_df[column][1] < 11:
        drop_list2.append(column)

In [10]:
# drop_list를 제거한 데이터 프레임을 가져온다.
single_cc_df = single_df.loc[:,single_df.columns.drop(drop_list)].copy()
multi_cc_df = multi_df.loc[:,multi_df.columns.drop(drop_list2)].copy()
print(single_cc_df.shape)
print(multi_cc_df.shape)

(2, 67)
(2, 59)


In [11]:
# 앞서 지워주었던 Singleplayer 컬럼의 값을 입력한다( 최초의 값과 동일하게 )
single_cc_df['Singleplayer'] = 104

In [12]:
# Singleplayer 컬럼(싱글 플레이어의 기준값)과 나머지 컬럼간의 기울기를 구한다.
np.polyfit(single_cc_df['Singleplayer'],single_cc_df.loc[:,single_cc_df.columns.drop(['Singleplayer'])], 1)[0] # 0은 기울기, 1은 절편

array([0.10096154, 0.10336538, 0.24038462, 0.37259615, 0.04567308,
       0.16105769, 0.04567308, 0.06971154, 0.12740385, 0.13221154,
       0.04567308, 0.10817308, 0.06009615, 0.05528846, 0.03365385,
       0.17067308, 0.09134615, 0.08653846, 0.0625    , 0.08173077,
       0.12740385, 0.11778846, 0.10096154, 0.10576923, 0.34134615,
       0.05769231, 0.13701923, 0.22115385, 0.16586538, 0.04086538,
       0.26682692, 0.15144231, 0.05288462, 0.08413462, 0.24519231,
       0.04326923, 0.04807692, 0.12980769, 0.11298077, 0.05769231,
       0.05048077, 0.06490385, 0.09134615, 0.07211538, 0.06490385,
       0.0625    , 0.05769231, 0.04807692, 0.04567308, 0.11538462,
       0.05288462, 0.04086538, 0.05288462, 0.25      , 0.07692308,
       0.11778846, 0.05288462, 0.08173077, 0.07211538, 0.05769231,
       0.04086538, 0.05048077, 0.06009615, 0.05288462, 0.0625    ,
       0.05048077, 0.05769231])

In [13]:
# 판매량/인기순위 간 상관관계 분석을 위해 데이터프레임의 행과 열을 바꾸어준다.
trans_single_cc_df = single_cc_df.transpose()

# 판매량/인기순위 간 상관관계분석
single_CC = np.corrcoef(trans_single_cc_df.iloc[:,0],trans_single_cc_df.iloc[:,1])
print(single_CC)

[[1.         0.85768677]
 [0.85768677 1.        ]]


In [14]:
# Multiplayer 컬럼값을 설정해준다.
multi_cc_df['Multiplayer'] = 105

In [15]:
# Multiplayer 컬럼(싱글 플레이어의 기준값)과 나머지 컬럼간의 기울기를 구한다.
np.polyfit(multi_cc_df['Multiplayer'],multi_cc_df.loc[:,multi_cc_df.columns.drop(['Multiplayer'])], 1)[0] # 0은 기울기, 1은 절편

array([0.16904762, 0.17857143, 0.06190476, 0.43333333, 0.08571429,
       0.11904762, 0.2547619 , 0.15238095, 0.27142857, 0.38095238,
       0.15      , 0.06190476, 0.07619048, 0.09285714, 0.09761905,
       0.06190476, 0.06428571, 0.11190476, 0.05952381, 0.08809524,
       0.18095238, 0.06666667, 0.16190476, 0.15952381, 0.06904762,
       0.25952381, 0.10238095, 0.08333333, 0.10238095, 0.23333333,
       0.17380952, 0.36666667, 0.03809524, 0.19285714, 0.12142857,
       0.05      , 0.10952381, 0.12142857, 0.0547619 , 0.08571429,
       0.05      , 0.07619048, 0.10238095, 0.05952381, 0.06428571,
       0.06428571, 0.08571429, 0.1047619 , 0.07619048, 0.05714286,
       0.05952381, 0.07857143, 0.08571429, 0.07380952, 0.04761905,
       0.0547619 , 0.04761905, 0.03571429, 0.03333333])

In [16]:
# 판매량/인기순위 간 상관관계 분석을 위해 데이터프레임의 행과 열을 바꾸어준다.
trans_multi_cc_df = multi_cc_df.transpose()

# 판매량/인기순위 간 상관관계분석
multi_CC = np.corrcoef(trans_multi_cc_df.iloc[:,0],trans_multi_cc_df.iloc[:,1])
print(multi_CC)

[[1.         0.94637776]
 [0.94637776 1.        ]]


In [17]:
# 컬럼 간 상관도가 높은 컬럼을 출력한다
x = np.polyfit(multi_cc_df['Multiplayer'],multi_cc_df.loc[:,multi_cc_df.columns.drop(['Multiplayer'])], 1)[0]
for i, n in enumerate(x):
    if n > 0.2:
        print(f'{i} "{multi_cc_df.columns[i]}" 카테고리의 상관관계 값 {n}')
        

3 "Action" 카테고리의 상관관계 값 0.4333333333333332
6 "First-Person" 카테고리의 상관관계 값 0.2547619047619048
8 "Online Co-Op" 카테고리의 상관관계 값 0.2714285714285714
9 "Co-op" 카테고리의 상관관계 값 0.38095238095238093
25 "Adventure" 카테고리의 상관관계 값 0.2595238095238095
29 "Open World" 카테고리의 상관관계 값 0.2333333333333334
31 "Singleplayer" 카테고리의 상관관계 값 0.3666666666666667


In [18]:
# 기울기 값
x = np.polyfit(single_cc_df['Singleplayer'],single_cc_df.loc[:,single_cc_df.columns.drop(['Singleplayer'])], 1)[0]
for i, n in enumerate(x):
    if n > 0.2:
        print(f'{i} "{single_cc_df.columns[i]}" 카테고리의 상관관계 값 {n}')

2 "Multiplayer" 카테고리의 상관관계 값 0.2403846153846153
3 "Action" 카테고리의 상관관계 값 0.3725961538461538
24 "Adventure" 카테고리의 상관관계 값 0.34134615384615374
27 "Open World" 카테고리의 상관관계 값 0.22115384615384617
30 "Atmospheric" 카테고리의 상관관계 값 0.2668269230769231
34 "Great Soundtrack" 카테고리의 상관관계 값 0.24519230769230774
53 "Story Rich" 카테고리의 상관관계 값 0.25


### 피어슨

In [19]:
import scipy.stats as stats
stats.pearsonr(single_cc_df.loc[:,single_cc_df.columns.drop(['Singleplayer'])],single_cc_df['Singleplayer'])

PearsonRResult(statistic=nan, pvalue=nan)